# Python project starter with sqlite3
## Concepts to study:
* Database connection
* sqlite3 data types allowed
 * How do they differ from other common database engines?
* Pandas dataframes
* Data insert with to_sql
* Fetch rows from database
* Database triggers

## Ideas to extend this project:
* Save the database historical_places.db to your google drive so you can use it later
* Come up with a definition of protected
 * UNESCO heritage site? Redacted street address?   
* What if we change place name to unique and not null?
* What other triggers might we want? Do we need to audit any specific data items that might change?
* What if we add a username to the database trigger? $USER on *nix or %username% on windows
* Insert more rows of data to the existing database
* Find a real dataset, like the historical places from the National Park Service, and modify the below to use it
 * What dependency does pandas.read_excel need?
* Get a free cloud account and do this in the cloud


In [1]:
import pandas as pd
import sqlite3
!ls

sample_data


In [3]:
#variables
database_file = 'historical_places.db'
state_DC = "DISTRICT OF COLUMBIA"

In [4]:
#input data
data = {
    'id': [1, 2, 3, 4],
    'place_name': ['Parthenon', 'Jupiter', 'Lake Nemi', 'Athens'],
    'street & number': ['123 Main St', '456 Oak Ave', '789 Elm Rd', '101 Pine Blvd'],
    'protected': [True, False, True, False]
}

df = pd.DataFrame(data)
print(df)

   id place_name street & number  protected
0   1  Parthenon     123 Main St       True
1   2    Jupiter     456 Oak Ave      False
2   3  Lake Nemi      789 Elm Rd       True
3   4     Athens   101 Pine Blvd      False


In [5]:
# create table sql code
create_historical_places_table = """ create table if not exists historical_places (
        id integer primary key not null,
        PlaceName text not null,
        PlaceAddress text,
        Protected text,
        CreateDate default ( datetime('now') ),
        UpdateDate default ( datetime('now') )
        );"""

In [6]:
#test insert
insert_row = """insert into historical_places
                          (PlaceName, PlaceAddress, Protected)
                           values ('Delphi','345 Pythia Pl','True')"""

In [7]:
# an interesting check you should know about
check_tables = "select name from sqlite_master where type='table'"

In [8]:
# create trigger sql code
# this trigger puts an datetime stamp when the row was updated.
create_trigger_audit_update_date = """ create trigger if not exists audit_update_date after update on historical_places
begin
  update historical_places set UpdateDate = datetime('now');
end;"""

In [9]:
# rename variables to match sql column names
df_to_sql_names = {
        # id is same
        'place_name' : 'PlaceName' ,
        'City ' : 'City' ,
        'street & number' : 'PlaceAddress' ,
        'protected' : 'Protected' ,
        }

In [10]:
# database connection - 'with' handles some exceptions but doesn't close it for you
with sqlite3.connect(database_file) as conn:
    cursor = conn.cursor()
    # historical_places table:
    #    create table
    cursor.execute(create_historical_places_table)
    conn.commit()
#    res = cursor.execute("""select 'a' as now, * from historical_places""")
#    for a in res.fetchall():
#      print("row=", a)
    #    trigger
    cursor.execute(create_trigger_audit_update_date)
    conn.commit()

In [11]:
#ls lists what's in the current working directory
!ls
#do you see historical_places.db? good!

historical_places.db  sample_data


In [12]:
#another way to check is this:
r = cursor.execute(check_tables)
print(r.fetchall())

[('historical_places',)]


In [13]:
# rename columns in dataframe
new_df = df.rename(columns=df_to_sql_names)
print(new_df)
# to_sql insert to database
new_df.to_sql('historical_places', con= conn, if_exists='append', index=False) #specifying index=False results in an exception

   id  PlaceName   PlaceAddress  Protected
0   1  Parthenon    123 Main St       True
1   2    Jupiter    456 Oak Ave      False
2   3  Lake Nemi     789 Elm Rd       True
3   4     Athens  101 Pine Blvd      False


4

In [14]:
# try using print() to print more than one output in a notebook
res = cursor.execute("select * from historical_places")
for r in res.fetchall():
    print(r)
cursor.execute(insert_row)
conn.commit()

(1, 'Parthenon', '123 Main St', '1', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(2, 'Jupiter', '456 Oak Ave', '0', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(3, 'Lake Nemi', '789 Elm Rd', '1', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(4, 'Athens', '101 Pine Blvd', '0', '2023-11-29 23:08:41', '2023-11-29 23:08:41')


In [15]:
res = cursor.execute("select * from historical_places")
for r in res.fetchall():
    print(r)

#make sure you close database connections
conn.close()

(1, 'Parthenon', '123 Main St', '1', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(2, 'Jupiter', '456 Oak Ave', '0', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(3, 'Lake Nemi', '789 Elm Rd', '1', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(4, 'Athens', '101 Pine Blvd', '0', '2023-11-29 23:08:41', '2023-11-29 23:08:41')
(5, 'Delphi', '345 Pythia Pl', 'True', '2023-11-29 23:09:47', '2023-11-29 23:09:47')
